In [1]:
import pandas as pd
import numpy as np
import tensorflow

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, LSTM

2025-06-14 12:31:31.192002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749904291.405647      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749904291.466681      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
batch_size = 64
epochs= 100
latent_dim = 256
num_samples = 20000

In [3]:
data_path = '/kaggle/input/spanish-english/spa.txt'

data_path

'/kaggle/input/spanish-english/spa.txt'

# Vectorize the data

In this step we are reading all the unique characters from input text (i.e english sentence) and also from output text (i.e spanish sentence). And also separating english sentences to input_texts and spanish sentences to output_texts as basically our input will be english sentences and output will be spanish translation of that english sentences, or words.

The input_characters consist of all the unique characters present in input_texts( english sentences) and similarly output_characters consist of all the unique characters present in output_texts (spanish sentences).

In [4]:
input_texts = []
output_texts = []
input_characters = set()
output_characters = set()

with open(data_path, 'r', encoding= 'utf-8') as f:
    lines = f.read().split('\n')

for line in lines[:min(num_samples, len(lines)-1)]:
    input_text, output_text, _ = line.split('\t')

    # We use 'tab('\t')' as the 'start sequence' character For the targets,
    # and '\n' as the 'end sequence' charcter

    output_text = '\t' + output_text + '\n'
    input_texts.append(input_text)
    output_texts.append(output_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in output_text:
        if char not in output_characters:
            output_characters.add(char)

In [5]:
len(output_characters)

86

# Assigning Variables

* input_characters consist of a sorted list of all the unique characters present in english text.
* Ouput_character consists of a sorted list of all the characters present in french text.
* num_encoder_tokens and num_decoder_tokens consist of the length of input_characters and output_characters respectively.
* Max_encoder_seq_length and max_decoder_seq_length consist of maximum length of sentence present in the english text and maximum length of sentence in the spanish text respectively.

In [6]:
input_characters = sorted(list(input_characters))
output_characters = sorted(list(output_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(output_characters)

max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in output_texts])

In [7]:
print("Number of Samples:", len(input_texts))
print('Number of unique input Tokens:' , num_encoder_tokens)
print('Number of unique output Tokens:' , num_decoder_tokens)
print('Max sequence length for inputs:' , max_encoder_seq_length)
print('Max sequence length for outputs:' , max_decoder_seq_length)

Number of Samples: 20000
Number of unique input Tokens: 72
Number of unique output Tokens: 86
Max sequence length for inputs: 19
Max sequence length for outputs: 49


# Assigning Tokens

In [8]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])

output_token_index = dict([(char, i) for i, char in enumerate(output_characters)])

In [9]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 'é': 71}

In [10]:
output_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '¡': 71,
 '«': 72,
 '»': 73,
 '¿': 74,
 'Á': 75,
 'É': 76,
 'Ó': 77,
 'Ú': 78,
 'á': 79,
 'é': 80,
 'í': 81,
 'ñ': 82,
 'ó': 83,
 'ú': 84,
 'ü': 85}

# Turn the sentences into 3 numpy arrays

* encoder_input_data will be a 3D array of shape( num_pairs, max_english_sentence_length, num_english_characters) containing one-hot vectorization of the English sentences.
* decoder_input_data will also be a 3D array of shape( num_pairs, max_spanish_sentence_length, num_spanish_characters) containing one-hot vectorization of the Spanish sentences.
* decoder_output_data is same as decoder_input_data but offset by one timestep,
* decoder_output_data[ : , t , : ] will be same as decoder_input_data[ : , t+1 , : ].

In [11]:
'''
Creating a 3D array using np.zeros() method in which,

num_pairs ⬅⬅⬅👈 length of input_texts
max_english_sentence_length ⬅⬅⬅👈 max_encoder_seq_length
max_spanish_sentence_length ⬅⬅⬅👈 max_decoder_seq_length
num_english_characters ⬅⬅⬅👈 num_encoder_tokens
num_french_characters ⬅⬅⬅👈 num_decoder_tokens
'''

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length,
                              num_encoder_tokens), dtype= 'float32')

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length,
                              num_decoder_tokens), dtype= 'float32')

decoder_output_data = np.zeros((len(input_texts), max_decoder_seq_length,
                               num_decoder_tokens), dtype= 'float32')

# One-hot Vectorization

In [12]:
print(f"Shape of encoder_input_data: {encoder_input_data.shape}")
print(f"Number of input texts: {len(input_texts)}")
print(f"Number of output texts: {len(output_texts)}")

for i,(input_text, output_text) in enumerate(zip(input_texts, output_texts)):
    
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1

    for t, char in enumerate(output_text):
        # decoder_output_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, output_token_index[char]] = 1
        if t>0:
            # decoder_output_data will be ahead by one timestep
            # and will not include the start character
            decoder_output_data[i, t-1, output_token_index[char]] = 1

    decoder_input_data[i, t+1:, output_token_index[' ']] = 1
    decoder_output_data[i, t:, output_token_index[' ']] = 1

Shape of encoder_input_data: (20000, 19, 72)
Number of input texts: 20000
Number of output texts: 20000


In [13]:
encoder_input_data[0].shape

(19, 72)

# Creating LSTM layer

## LSTM layer for encoder

In [14]:
# Define an input sequence and process it:

encoder_inputs = Input(shape= (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard 'encoder_outputs' and only keep the states

encoder_states = [state_h, state_c]

I0000 00:00:1749904306.866000      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


## LSTM layer for decoder

In [15]:
# Set up the decoder, using encoder_states as initial states:

decoder_inputs = Input(shape= (None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences= True, return_state= True)

# We set our decoder to return full output sequences, and to return internal states
# as well but we don't use this internal states in the training model,
# but we will use them in inference.

decoder_outputs, _ , _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation= 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Creating a Model

In [16]:
# Define a model that will turn 'encoder_input_data' & 'decoder_input_data'
# into 'decoder_output_data'

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training:

model.compile(optimizer= 'rmsprop', loss= 'categorical_crossentropy',
              metrics= ['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_output_data,
         batch_size= batch_size, epochs= epochs, validation_split= 0.2)

Epoch 1/100


I0000 00:00:1749904312.240514      59 cuda_dnn.cc:529] Loaded cuDNN version 90300


250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.6713 - loss: 1.5332 - val_accuracy: 0.6654 - val_loss: 1.3692
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7272 - loss: 0.9863 - val_accuracy: 0.7013 - val_loss: 1.0091
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7578 - loss: 0.8245 - val_accuracy: 0.7230 - val_loss: 0.9246
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7750 - loss: 0.7570 - val_accuracy: 0.7330 - val_loss: 0.8827
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7834 - loss: 0.7201 - val_accuracy: 0.7426 - val_loss: 0.8419
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7916 - loss: 0.6912 - val_accuracy: 0.7554 - val_loss: 0.8024
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8010 - loss: 0.6591 - val_accuracy: 0.7635 - val_loss: 0.7730
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8094 - loss: 0.6332 - val_accuracy

# Creating Inferencing (Sampling)

In [17]:
# define sampling models:

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape= (latent_dim, ))
decoder_state_input_c = Input(shape= (latent_dim, ))

decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state= decoder_input_states)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_input_states, [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back
# to something readable

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())

reverse_output_char_index = dict((i, char) for char, i in output_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    output_seq = np.zeros((1, 1, num_decoder_tokens))
    output_seq[0, 0, output_token_index['\t']] = 1

    stop_condition = False
    decoded_sentences = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([output_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_output_char_index[sampled_token_index]
        decoded_sentences += sampled_char

        if(sampled_char == '\n' or len(decoded_sentences) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1):
        output_seq = np.zeros((1, 1, num_decoder_tokens))
        output_seq[0, 0, sampled_token_index] = 1

        states_value = [h, c]
    return decoded_sentences

for seq_index in range(20):
    # Take one sequence for trying out decoding:

    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentences = decode_sequence(input_seq)
    print("English to Spanish Translation")
    print("Input sentence: ", input_texts[seq_index])
    print("Decoded sentence: ", decoded_sentences)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
English to Spanish Translation
Input sentence:  Go.
Decoded sentence:  Vete.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
English to Spanish Translation
Input sentence:  Go.
Decoded sentence:  Vete.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
English to Spanish Translation
Input sentenc